In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from ipywidgets import interact

In [8]:
class BernoulliNaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.class_priors = None
        self.feature_probs = None
        self.log_class_priors = None
        self.log_feature_probs = None
        self.log_feature_probs_neg = None
        self.n_classes = None

    def compute_priors(self, y):
        class_counts = np.bincount(y)
        class_priors = class_counts / len(y)
        return class_priors

    def compute_likelihood(self, X, y):
        n_classes = self.n_classes
        n_features = X.shape[1]
        feature_probs = np.zeros((n_classes, n_features))

        for c in range(n_classes):
            X_c = X[y == c]
            feature_probs[c, :] = (np.sum(X_c, axis=0) + self.alpha) / (X_c.shape[0] + 2 * self.alpha)

        return feature_probs

    def fit(self, X, y):
        self.n_classes = np.max(y) + 1
        self.class_priors = self.compute_priors(y)
        self.feature_probs = self.compute_likelihood(X, y)
        self.log_class_priors = np.log(self.class_priors)
        self.log_feature_probs = np.log(self.feature_probs)
        self.log_feature_probs_neg = np.log(1 - self.feature_probs)

    def predict_log_proba(self, X):
        # Vectorized computation of log-probabilities
        log_probs = (
            self.log_class_priors[np.newaxis, :] +
            X @ self.log_feature_probs.T +
            (1 - X) @ self.log_feature_probs_neg.T
        )
        return log_probs

    def predict(self, X):
        log_probs = self.predict_log_proba(X)
        pred = np.argmax(log_probs, axis=1)
        return pred

    def accuracy(self, X, y):
        y_pred = self.predict(X)
        accuracy = np.mean(y_pred == y)
        return accuracy

In [9]:
# Load MNIST dataset
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"].astype(np.int8)

# Split into training and test sets
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

# Binarize the images
threshold = 128  # Threshold for binarization
X_train_bin = (X_train >= threshold).astype(np.int8)
X_test_bin = (X_test >= threshold).astype(np.int8)

# Create and train the classifier
alpha = 1  # Laplace smoothing parameter
nb_classifier = BernoulliNaiveBayes(alpha=alpha)
print("Training the Bernoulli Naive Bayes classifier...")
nb_classifier.fit(X_train_bin, y_train)

# Evaluate the classifier
print("Evaluating the classifier on the test set...")
accuracy = nb_classifier.accuracy(X_test_bin, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

Loading MNIST dataset...
Training the Bernoulli Naive Bayes classifier...
Evaluating the classifier on the test set...
Accuracy: 84.27%


In [10]:
# Get the predicted labels on the test set
y_pred_test = nb_classifier.predict(X_test_bin)

def show_test_sample(index):
    """
    Displays the test image at the given index, along with its true and predicted labels.
    """
    image = X_test[index].reshape(28, 28)  # Use original grayscale image for better visualization
    true_label = y_test[index]
    predicted_label = y_pred_test[index]
    
    plt.figure(figsize=(4, 4))
    plt.imshow(image, cmap='gray')
    plt.title(f"Index: {index}\nTrue Label: {true_label} | Predicted Label: {predicted_label}")
    plt.axis('off')
    plt.show()

# Create an interactive widget to browse through test samples
print("Use the slider to browse through test images.")
interact(show_test_sample, index=(0, len(X_test) - 1))

Use the slider to browse through test images.


interactive(children=(IntSlider(value=4999, description='index', max=9999), Output()), _dom_classes=('widget-i…

<function __main__.show_test_sample(index)>